# Next Step


Discuss general algorithm: need to approximate gradient for back propagation. Then present gradient approximation methods.
- Closed-Form Decisions
- Linear Decision Objective
- Quadratic Decision Objective
- Generic Decision Objective

Gradient Free Methods

Experiments

Methods to compare:
- Two-stage: prediction then decision, prediction then fair decision, fair prediction then decision, fair prediction then fair decision
- DFL: DFL version of each of the above two-stage settings


Performance measures to report:
- Prediction accuracy: mean square errors of $r$ and $\hat{r}$
- Decision accuracy: mean square errors of $d(r)$ and $d(\hat{r})$
- Prediction fairness: prediction fairness measure of $\hat{r}$
- Decision fairness: decision fairness measure of $d(\hat(r))$
- Runtime of algorithm

In [1]:
import cvxpy as cp
import numpy as np
import warnings
import sys
from IPython.core.interactiveshell import InteractiveShell
import torch
import torch.optim as optim
from torch import nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader, TensorDataset, random_split
import matplotlib.pyplot as plt
import pandas as pd
import sys
# sys.path.insert(0, 'E:\\User\\Stevens\\Code\\The Paper\\algorithm')
from torch.utils.data import Dataset, DataLoader


import warnings
warnings.filterwarnings("ignore")

from myutil import *
from features import *

In [16]:
#df = pd.read_csv('data/data.csv')
df = pd.read_csv('data.csv')

columns_to_keep = [
    'risk_score_t', 'program_enrolled_t', 'cost_t', 'cost_avoidable_t', 'race', 'dem_female', 'gagne_sum_tm1', 'gagne_sum_t', 
    'risk_score_percentile', 'screening_eligible', 'avoidable_cost_mapped', 'propensity_score', 'g_binary', 
    'g_continuous', 'utility_binary', 'utility_continuous'
]
# for race 0 is white, 1 is black
df_stat = df[columns_to_keep]
df_feature = df[[col for col in df.columns if col not in columns_to_keep]]

# Replace all values less than 0.1 with 0.1
#df['risk_score_t'] = df['risk_score_t'].apply(lambda x: 0.1 if x < 0.1 else x)
df['g_continuous'] = df['g_continuous'].apply(lambda x: 0.1 if x < 0.1 else x)

# subset a sample of 5000 rows of df
# df = df.sample(n=10000, random_state=1)

df.shape

(48784, 168)

In [28]:
# Define input variables for DFL
feats = df[get_all_features(df)].values
risk = df['risk_score_t'].values
gainF = df['g_continuous'].values
pScore = df['propensity_score'].values
cost = np.ones(risk.shape)

alpha = 0.5
Q = 1000

utility = risk * gainF * pScore

risk.mean()

4.393691680358348

# Prediction Stage

In [8]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
feats = scaler.fit_transform(feats)

# Train the Prediction Model
class RiskDataset(Dataset):
    def __init__(self, features, risks):
        self.features = torch.FloatTensor(features)
        self.risks = torch.FloatTensor(risks).reshape(-1, 1)
        
    def __len__(self):
        return len(self.features)
        
    def __getitem__(self, idx):
        return self.features[idx], self.risks[idx]
    
class RiskPredictor(nn.Module):
    def __init__(self, input_dim):
        super().__init__()
        self.model = nn.Sequential(
            nn.Linear(input_dim, 1),

            nn.Softplus()
        )
    
    def forward(self, x):
        return self.model(x)

# Training function
def train_model(features, risks, epochs=10, batch_size=32):
    dataset = RiskDataset(features, risks)
    dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True)
    
    model = RiskPredictor(features.shape[1])
    model.train()
    criterion = nn.MSELoss()
    optimizer = torch.optim.Adam(model.parameters(), lr=0.0001)
    
    for epoch in range(epochs):
        for batch_features, batch_risks in dataloader:
            optimizer.zero_grad()
            predictions = model(batch_features)
            loss = criterion(predictions, batch_risks)
            loss.backward()
            optimizer.step()
            
        if (epoch + 1) % 5 == 0:
            print(f'Epoch [{epoch+1}/{epochs}], Loss: {loss.item():.4f}')
    
    return model



# model = train_model(feats, risk)
# torch.save(model.state_dict(), 'risk_predictor_model.pth')

# Load the model from local
model = RiskPredictor(feats.shape[1])
model.load_state_dict(torch.load('risk_predictor_model.pth'))
model.eval()

pred_risk = model(torch.FloatTensor(feats)).detach().numpy().flatten()

pred_risk.mean(), risk.mean()

(2.7172887, 4.394240983699624)

In [9]:
pred_risk.min()

0.0

# Solve Optimization Problem

In [27]:
def AlphaFairness(util,alpha):
    if alpha == 1:
        return np.sum(np.log(util))
    elif alpha == 0:
        return np.sum(util)
    elif alpha == 'inf':
        return np.min(util)
    else:
        return np.sum(util**(1-alpha)/(1-alpha))
    
def solve_optimization(gainF, risk, cost, alpha, Q):
    # Flatten input arrays
    gainF, risk, cost = gainF.flatten(), risk.flatten() + 0.001, cost.flatten()
    d = cp.Variable(risk.shape, nonneg=True)
    
    utils = cp.multiply(cp.multiply(gainF, risk), d)
    
    if alpha == 'inf':
        # Maximin formulation
        t = cp.Variable()  # auxiliary variable for minimum utility
        objective = cp.Maximize(t)
        constraints = [
            d >= 0,
            # d <= 1,
            cp.sum(cost * d) <= Q,
            utils >= t  # t is the minimum utility
        ]
    elif alpha == 1:
        # Nash welfare (alpha = 1)
        objective = cp.Maximize(cp.sum(cp.log(utils)))
        constraints = [
            d >= 0,
            # d <= 1,
            cp.sum(cost * d) <= Q
        ]
    elif alpha == 0:
        # Utilitarian welfare (alpha = 0)
        objective = cp.Maximize(cp.sum(utils))
        constraints = [
            d >= 0,
            # d <= 1,
            cp.sum(cost * d) <= Q
        ]
    else:
        # General alpha-fairness
        objective = cp.Maximize(cp.sum(utils**(1-alpha))/(1-alpha) if alpha != 0 
                              else cp.sum(utils))
        constraints = [
            d >= 0,
            # d <= 1,
            cp.sum(cost * d) <= Q
        ]
    
    # Solve the problem
    problem = cp.Problem(objective, constraints)
    problem.solve(solver=cp.MOSEK, verbose=True, warm_start=True, mosek_params={'MSK_IPAR_LOG': 1})
    
    if problem.status != 'optimal':
        print(f"Warning: Problem status is {problem.status}")
    
    optimal_decision = d.value
    optimal_value = AlphaFairness(optimal_decision * gainF * risk, alpha)
    
    return optimal_decision, optimal_value

In [30]:
# pred_sol,_ = solve_optimization(gainF, predicted_risk, cost, alpha='inf', Q=Q)
# pred_obj = np.sum((risk * gainF * pred_sol)**(1-alpha)/(1-alpha))
# true_obj = np.sum((optimal_decision * gainF * risk)**(1-alpha)/(1-alpha))

In [25]:
def twoStagePTO(model, feats, gainF, risk, cost, Q, alphas=[0.5]):
    """
    Perform a two-stage optimization analysis with predictions and calculate normalized regrets.

    Args:
        model (nn.Module): A regression neural network for risk prediction.
        feats (np.ndarray): Feature array for predictions.
        gainF (np.ndarray): Gain factors.
        risk (np.ndarray): True risk values.
        cost (np.ndarray): Cost constraints.
        Q (float): Budget constraint.
        alphas (list): List of alpha values for fairness.

    Returns:
        pd.DataFrame: A table of prediction risk means, true risk mean, objectives, and normalized regrets.
    """
    from sklearn.preprocessing import StandardScaler
    import pandas as pd

    # Feature scaling
    scaler = StandardScaler()
    feats_scaled = scaler.fit_transform(feats)

    # Predict risks
    model.eval()
    pred_risk = model(torch.FloatTensor(feats_scaled)).detach().numpy().flatten()

    # Initialize result storage
    results = []

    # Iterate over alphas
    for alpha in alphas:
        # Solve optimization problems
        true_sol, _ = solve_optimization(gainF, risk, cost, alpha, Q)
        pred_sol, _ = solve_optimization(gainF, pred_risk, cost, alpha, Q)

        # Calculate true and predicted utilities
        true_utility = (risk + 0.001) * gainF * true_sol
        pred_utility = (pred_risk + 0.001) * gainF * pred_sol
        pred_utility_truerisk = (risk + 0.001) * gainF * pred_sol

        # Calculate objectives
        true_obj = AlphaFairness(true_utility, alpha)
        pred_obj = AlphaFairness(pred_utility, alpha)
        pred_obj_truerisk = AlphaFairness(pred_utility_truerisk, alpha)

        # Calculate regret and normalized regret
        # regret = true_obj - pred_obj
        regret = true_obj - pred_obj_truerisk
        normalized_regret = regret / (abs(true_obj) + 1e-7)

        # Collect results for this alpha
        results.append({
            'Alpha': alpha,
            'Predicted Risk Mean': pred_risk.mean(),
            'True Risk Mean': risk.mean(),
            'True Objective': true_obj,
            'Predicted Objective': pred_obj,
            'Regret': regret,
            'Normalized Regret': normalized_regret
        })

    # Create a DataFrame for results
    results_df = pd.DataFrame(results)
    print(results_df)
    return results_df


In [29]:
data_sample = df.sample(n=40000, random_state=42)
feats_sample = data_sample[get_all_features(data_sample)].values
risk_sample = data_sample['risk_score_t'].values
gainF_sample = data_sample['g_continuous'].values
decision_sample = data_sample['propensity_score'].values
cost_sample = np.ones(risk_sample.shape)

twoStagePTO(model, feats_sample, gainF_sample, risk_sample, cost_sample, Q, alphas=[0,.5,.99,1,2,'inf'])

                                     CVXPY                                     
                                     v1.4.3                                    
(CVXPY) Jan 08 01:22:36 PM: Your problem has 40000 variables, 2 constraints, and 0 parameters.
(CVXPY) Jan 08 01:22:36 PM: It is compliant with the following grammars: DCP, DQCP
(CVXPY) Jan 08 01:22:36 PM: (If you need to solve this problem multiple times, but with different data, consider using parameters.)
(CVXPY) Jan 08 01:22:36 PM: CVXPY will first compile your problem; then, it will invoke a numerical solver to obtain a solution.
(CVXPY) Jan 08 01:22:36 PM: Your problem is compiled with the CPP canonicalization backend.
-------------------------------------------------------------------------------
                                  Compilation                                  
-------------------------------------------------------------------------------
(CVXPY) Jan 08 01:22:36 PM: Compiling problem (target solver=MOSEK).


(CVXPY) Jan 08 01:22:48 PM: 
(CVXPY) Jan 08 01:22:48 PM: 
(CVXPY) Jan 08 01:22:48 PM: Interior-point solution summary
(CVXPY) Jan 08 01:22:48 PM:   Problem status  : PRIMAL_AND_DUAL_FEASIBLE
(CVXPY) Jan 08 01:22:48 PM:   Solution status : OPTIMAL
(CVXPY) Jan 08 01:22:48 PM:   Primal.  obj: -7.9602358089e+06   nrm: 8e+03    Viol.  con: 5e-13    var: 0e+00  
(CVXPY) Jan 08 01:22:48 PM:   Dual.    obj: -7.9602358089e+06   nrm: 1e+03    Viol.  con: 0e+00    var: 0e+00  
-------------------------------------------------------------------------------
                                    Summary                                    
-------------------------------------------------------------------------------
(CVXPY) Jan 08 01:22:48 PM: Problem status: optimal
(CVXPY) Jan 08 01:22:48 PM: Optimal value: 7.960e+06
(CVXPY) Jan 08 01:22:48 PM: Compilation took 3.727e-02 seconds
(CVXPY) Jan 08 01:22:48 PM: Solver (including time spent in interface) took 6.096e+00 seconds
                           

(CVXPY) Jan 08 01:22:49 PM: 17  3.4e-06  3.4e-03  1.3e-03  9.40e-01   -7.362957429e+04  -7.362882337e+04  3.4e-06  1.06  
(CVXPY) Jan 08 01:22:49 PM: 18  1.2e-06  1.2e-03  3.1e-04  9.79e-01   -7.362965097e+04  -7.362936999e+04  1.2e-06  1.10  
(CVXPY) Jan 08 01:22:49 PM: 19  9.2e-07  9.2e-04  2.1e-04  8.96e-01   -7.362958241e+04  -7.362936400e+04  9.2e-07  1.14  
(CVXPY) Jan 08 01:22:49 PM: 20  3.4e-07  3.4e-04  4.7e-05  9.64e-01   -7.362952839e+04  -7.362944651e+04  3.4e-07  1.19  
(CVXPY) Jan 08 01:22:49 PM: 21  2.4e-07  2.4e-04  3.1e-05  8.28e-01   -7.362948020e+04  -7.362941624e+04  2.4e-07  1.23  
(CVXPY) Jan 08 01:22:49 PM: 22  8.8e-08  8.8e-05  6.9e-06  9.39e-01   -7.362942736e+04  -7.362940365e+04  8.8e-08  1.27  
(CVXPY) Jan 08 01:22:49 PM: 23  6.0e-08  6.0e-05  4.4e-06  7.34e-01   -7.362939852e+04  -7.362938009e+04  6.0e-08  1.32  
(CVXPY) Jan 08 01:22:49 PM: 24  2.1e-08  2.1e-05  9.5e-07  9.05e-01   -7.362936373e+04  -7.362935697e+04  2.1e-08  1.36  
(CVXPY) Jan 08 01:22:50 

(CVXPY) Jan 08 01:22:51 PM: 2   7.1e-02  7.1e+01  2.7e+03  4.14e-02   -4.507854098e+04  -3.555880442e+04  7.1e-02  0.40  
(CVXPY) Jan 08 01:22:51 PM: 3   3.5e-02  3.5e+01  1.0e+03  4.67e-01   -4.092898372e+04  -3.543854948e+04  3.5e-02  0.45  
(CVXPY) Jan 08 01:22:51 PM: 4   2.6e-02  2.6e+01  6.8e+02  6.66e-01   -3.975749353e+04  -3.543597332e+04  2.6e-02  0.49  
(CVXPY) Jan 08 01:22:51 PM: 5   1.6e-02  1.6e+01  3.4e+02  7.09e-01   -3.845268836e+04  -3.560569666e+04  1.6e-02  0.53  
(CVXPY) Jan 08 01:22:51 PM: 6   1.5e-02  1.5e+01  3.1e+02  7.60e-01   -3.835967483e+04  -3.566458695e+04  1.5e-02  0.58  
(CVXPY) Jan 08 01:22:51 PM: 7   1.0e-02  1.0e+01  1.8e+02  7.58e-01   -3.789460340e+04  -3.595036045e+04  1.0e-02  0.62  
(CVXPY) Jan 08 01:22:51 PM: 8   6.1e-03  6.1e+00  9.2e+01  7.78e-01   -3.753176828e+04  -3.627426744e+04  6.1e-03  0.67  
(CVXPY) Jan 08 01:22:51 PM: 9   5.2e-03  5.2e+00  7.3e+01  7.80e-01   -3.750916318e+04  -3.641695402e+04  5.2e-03  0.71  
(CVXPY) Jan 08 01:22:51 

(CVXPY) Jan 08 01:22:58 PM: 70  9.7e-10  9.4e-05  1.7e-08  8.15e-01   -3.811416843e+04  -3.811416769e+04  6.0e-10  7.54  
(CVXPY) Jan 08 01:22:58 PM: 71  9.8e-10  9.4e-05  1.7e-08  8.07e-01   -3.811416842e+04  -3.811416768e+04  6.0e-10  7.71  
(CVXPY) Jan 08 01:22:59 PM: 72  9.8e-10  9.4e-05  1.7e-08  8.15e-01   -3.811416839e+04  -3.811416765e+04  6.0e-10  7.87  
(CVXPY) Jan 08 01:22:59 PM: 73  9.6e-10  9.3e-05  1.7e-08  8.16e-01   -3.811416833e+04  -3.811416759e+04  6.0e-10  8.02  
(CVXPY) Jan 08 01:22:59 PM: 74  9.2e-10  8.6e-05  1.5e-08  8.14e-01   -3.811416735e+04  -3.811416667e+04  5.4e-10  8.15  
(CVXPY) Jan 08 01:22:59 PM: 75  9.3e-10  8.2e-05  1.4e-08  8.07e-01   -3.811416689e+04  -3.811416624e+04  5.2e-10  8.30  
(CVXPY) Jan 08 01:22:59 PM: 76  9.2e-10  8.2e-05  1.4e-08  8.06e-01   -3.811416686e+04  -3.811416621e+04  5.2e-10  8.47  
(CVXPY) Jan 08 01:22:59 PM: 77  9.3e-10  8.2e-05  1.4e-08  8.07e-01   -3.811416683e+04  -3.811416619e+04  5.2e-10  8.63  
(CVXPY) Jan 08 01:22:59 

...

(CVXPY) Jan 08 01:23:33 PM: Factor     - dense dim.             : 1                 flops                  : 1.16e+06        
(CVXPY) Jan 08 01:23:33 PM: ITE PFEAS    DFEAS    GFEAS    PRSTATUS   POBJ              DOBJ              MU       TIME  
(CVXPY) Jan 08 01:23:33 PM: 0   1.0e+00  1.0e+03  1.0e+00  0.00e+00   -0.000000000e+00  -0.000000000e+00  1.0e+00  0.41  
(CVXPY) Jan 08 01:23:33 PM: 1   7.7e-01  7.7e+02  7.8e-01  -6.98e-01  3.027030720e+03   3.026910058e+03   7.7e-01  0.46  
(CVXPY) Jan 08 01:23:33 PM: 2   2.5e-01  2.5e+02  3.2e-01  -5.98e-01  2.798996855e+04   2.798900713e+04   2.5e-01  0.51  
(CVXPY) Jan 08 01:23:33 PM: 3   1.3e-01  1.3e+02  1.7e-01  -2.67e-01  5.051154818e+04   5.051025450e+04   1.3e-01  0.55  
(CVXPY) Jan 08 01:23:34 PM: 4   4.6e-02  4.6e+01  5.4e-02  1.31e-02   9.203981060e+04   9.203868713e+04   4.6e-02  0.60  
(CVXPY) Jan 08 01:23:34 PM: 5   3.2e-02  3.2e+01  3.4e-02  3.77e-01   1.076240822e+05   1.076231617e+05   3.2e-02  0.65  
(CVXPY) Jan 08 01:23

(CVXPY) Jan 08 01:23:36 PM: Eliminator terminated.
(CVXPY) Jan 08 01:23:36 PM: Linear dependency checker started.
(CVXPY) Jan 08 01:23:36 PM: Linear dependency checker terminated.
(CVXPY) Jan 08 01:23:36 PM: Eliminator started.
(CVXPY) Jan 08 01:23:36 PM: Freed constraints in eliminator : 0
(CVXPY) Jan 08 01:23:36 PM: Eliminator terminated.
(CVXPY) Jan 08 01:23:36 PM: Eliminator - tries                  : 2                 time                   : 0.00            
(CVXPY) Jan 08 01:23:36 PM: Lin. dep.  - tries                  : 1                 time                   : 0.01            
(CVXPY) Jan 08 01:23:36 PM: Lin. dep.  - primal attempts        : 1                 successes              : 1               
(CVXPY) Jan 08 01:23:36 PM: Lin. dep.  - dual attempts          : 0                 successes              : 0               
(CVXPY) Jan 08 01:23:36 PM: Lin. dep.  - primal deps.           : 0                 dual deps.             : 0               
(CVXPY) Jan 08 01:23:36 PM:

-------------------------------------------------------------------------------
                                  Compilation                                  
-------------------------------------------------------------------------------
(CVXPY) Jan 08 01:23:39 PM: Compiling problem (target solver=MOSEK).
(CVXPY) Jan 08 01:23:39 PM: Reduction chain: FlipObjective -> Dcp2Cone -> CvxAttr2Constr -> ConeMatrixStuffing -> MOSEK
(CVXPY) Jan 08 01:23:39 PM: Applying reduction FlipObjective
(CVXPY) Jan 08 01:23:39 PM: Applying reduction Dcp2Cone
(CVXPY) Jan 08 01:23:39 PM: Applying reduction CvxAttr2Constr
(CVXPY) Jan 08 01:23:39 PM: Applying reduction ConeMatrixStuffing
(CVXPY) Jan 08 01:23:39 PM: Applying reduction MOSEK
(CVXPY) Jan 08 01:23:39 PM: Finished problem compilation (took 4.320e-02 seconds).
-------------------------------------------------------------------------------
                                Numerical solver                               
------------------------------

(CVXPY) Jan 08 01:23:39 PM: Applying reduction CvxAttr2Constr
(CVXPY) Jan 08 01:23:39 PM: Applying reduction ConeMatrixStuffing
(CVXPY) Jan 08 01:23:39 PM: Applying reduction MOSEK
(CVXPY) Jan 08 01:23:39 PM: Finished problem compilation (took 4.434e-02 seconds).
-------------------------------------------------------------------------------
                                Numerical solver                               
-------------------------------------------------------------------------------
(CVXPY) Jan 08 01:23:39 PM: Invoking solver MOSEK  to obtain a solution.


(CVXPY) Jan 08 01:23:39 PM: Problem
(CVXPY) Jan 08 01:23:39 PM:   Name                   :                 
(CVXPY) Jan 08 01:23:39 PM:   Objective sense        : maximize        
(CVXPY) Jan 08 01:23:39 PM:   Type                   : LO (linear optimization problem)
(CVXPY) Jan 08 01:23:39 PM:   Constraints            : 40001           
(CVXPY) Jan 08 01:23:39 PM:   Affine conic cons.     : 0               
(CVXPY) J

,Alpha,Predicted Risk Mean,True Risk Mean,True Objective,Predicted Objective,Regret,Normalized Regret
0,0,2.703654,4.387568,2.841649e+05,7.960236e+06,1.047215e+05,3.685237e-01
1,0.5,2.703654,4.387568,7.362928e+04,3.811415e+04,2.388796e+04,3.244356e-01
2,0.99,2.703654,4.387568,3.955807e+06,3.889371e+06,6.562198e+00,1.658877e-06
3,1,2.703654,4.387568,-4.490093e+04,-1.127005e+05,-3.057545e-03,-6.809536e-08
4,2,2.703654,4.387568,-8.349522e+05,-1.546416e+06,2.847300e+06,3.410135e+00
5,inf,2.703654,4.387568,6.267983e-04,3.833028e-03,6.243504e-04,9.959358e-01


regrets

# SPO